<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=99972031e23ed2ed3fcd92a0e58c05079a5b7fe2ca306977f9c50273da5c3d66
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 13:17:33
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 46.18 K (0.32%)
Current PnL: -21.32 L (-13.97%)
CY Booked + Current PnL: -8.67 L (-5.68%)
-------------------
Total profit:  2.26 L
Total loss:  -23.58 L
-------------------
Total Booked + Current PnL: 18.31 L (14.58%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.83%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.29 L (61.63%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.53,65.0,X-LC,6.76,233516.0,23150.0,5628.0,1.28,11.00,2.41,13.68,37.0,4.11,1.64,29.35,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.07,50.0,M-SC,2.00,86630.0,-14147.0,14242.0,-1.06,-14.04,16.44,0.09,245.0,-0.99,0.61,13.81,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.45,61.0,X-MC,6.30,150723.0,14590.0,15992.0,0.82,10.72,10.61,22.47,91.0,0.91,1.06,25.09,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.65,44.0,X-MC,3.06,88185.0,-2493.0,17399.0,2.12,-2.75,19.73,16.44,101.0,-0.14,0.62,16.87,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.0,115.40,71.0,M-SC,5.88,141087.0,-33472.0,33508.0,13.32,-19.17,23.75,0.02,221.0,-1.00,0.99,22.72,OX40N,NTT,PAINTS
61,SAIL,228.0,58.78,71.0,M-MC,10.05,250327.0,25365.0,142286.0,3.10,11.28,56.84,74.53,192.0,0.18,1.76,47.17,XY24,BTT,STEEL
39,INFY,2275.0,-17.09,61.0,X-LC,4.86,323504.0,10616.0,161073.0,2.84,3.39,49.79,54.87,3.0,0.07,2.28,11.96,X40,BTT,IT
19,COFFEEDAY,80.0,-47.69,42.0,L-SC,27.35,76915.0,-36634.0,76684.0,2.74,-32.26,99.70,35.27,268.0,-0.48,0.54,88.25,XR,NTT,HOTELS
47,LAOPALA,464.0,99.12,46.0,H-SC,3.79,91054.0,-40051.0,91300.0,2.21,-30.55,100.27,39.09,142.0,-0.44,0.64,17.27,AR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.00,51.32,38.0,H-SC,14.53,139689.0,-39382.0,105116.0,-4.42,-21.99,75.25,36.71,135.0,-0.37,0.98,38.60,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.43,45.0,M-SC,7.02,119342.0,4544.0,89495.0,-3.19,3.96,74.99,81.92,223.0,0.05,0.84,41.28,XR,NTT,DURABLES
65,SHALBY,327.00,1064.27,37.0,M-SC,17.17,158302.0,-22841.0,68307.0,-3.01,-12.61,43.15,25.10,235.0,-0.33,1.11,26.86,XY24,NTT,HEALTHCARE
25,GREENPANEL,537.00,203.62,40.0,M-SC,5.25,135615.0,-47463.0,126434.0,-2.58,-25.93,93.23,43.14,230.0,-0.38,0.95,27.56,XY24,NTT,MISC
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.47,123757.0,-1540.0,60468.0,-2.40,-1.23,48.86,47.03,141.0,-0.03,0.87,54.96,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,50.0,M-SC,2.00,86630.0,-14147.0,14242.0,-1.06,-14.04,16.44,0.09,245.0,-0.99,0.61,13.81,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,47.0,H-SC,2.38,222768.0,-46899.0,83226.0,0.49,-17.39,37.36,13.47,138.0,-0.56,1.57,13.27,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,115.40,71.0,M-SC,5.88,141087.0,-33472.0,33508.0,13.32,-19.17,23.75,0.02,221.0,-1.00,0.99,22.72,OX40N,NTT,PAINTS
67,SIS,528.0,2043.01,50.0,H-SC,2.29,86483.0,-24549.0,48154.0,1.97,-22.11,55.68,21.26,156.0,-0.51,0.61,16.85,OX40N,NTT,MISC
17,CERA,9475.0,-25.06,39.0,H-SC,1.86,136436.0,-39467.0,81493.0,-1.08,-22.44,59.73,23.89,149.0,-0.48,0.96,17.83,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,58.78,71.0,M-MC,10.05,250327.0,25365.0,142286.0,3.10,11.28,56.84,74.53,192.0,0.18,1.76,47.17,XY24,BTT,STEEL
31,HINDZINC,730.22,29.16,53.0,M-LC,9.41,210758.0,5682.0,106896.0,2.11,2.77,50.72,54.89,52.0,0.05,1.48,27.53,X5K,ATH,METALS
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.43,45.0,M-SC,7.02,119342.0,4544.0,89495.0,-3.19,3.96,74.99,81.92,223.0,0.05,0.84,41.28,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.24,129119.0,5783.0,125839.0,-0.46,4.69,97.46,106.72,119.0,0.05,0.91,29.16,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-53.71,50.0,H-SC,5.24,129119.0,5783.0,125839.0,-0.46,4.69,97.46,106.72,119.0,0.05,0.91,29.16,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-42.43,45.0,M-SC,7.02,119342.0,4544.0,89495.0,-3.19,3.96,74.99,81.92,223.0,0.05,0.84,41.28,XR,NTT,DURABLES
31,HINDZINC,730.22,29.16,53.0,M-LC,9.41,210758.0,5682.0,106896.0,2.11,2.77,50.72,54.89,52.0,0.05,1.48,27.53,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,46.0,M-LC,5.59,148874.0,-2071.0,111522.0,1.53,-1.37,74.91,72.51,53.0,-0.02,1.05,4.57,XR,NTT,IT
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.47,123757.0,-1540.0,60468.0,-2.40,-1.23,48.86,47.03,141.0,-0.03,0.87,54.96,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.90,28.0,X-LC,7.17,201490.0,-8917.0,68083.0,0.47,-4.24,33.79,28.12,38.0,-0.13,1.42,18.39,X40N,ATH,FMCG
44,JSWINFRA,342.00,-23.96,30.0,X-MC,4.44,191025.0,-9322.0,39829.0,1.00,-4.65,20.85,15.23,178.0,-0.23,1.34,26.10,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-13.94,33.0,X-LC,6.23,241700.0,-15363.0,50491.0,0.12,-5.98,20.89,13.67,24.0,-0.30,1.70,14.10,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.60,34.0,X-MC,0.29,197910.0,-2910.0,70713.0,0.15,-1.45,35.73,33.76,80.0,-0.04,1.39,3.67,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,0.81,198583.0,-1555.0,22440.0,0.51,-0.78,11.30,10.44,4.0,-0.07,1.40,4.74,X40,NTT,FMCG
66,SIEMENS,4671.50,-4.97,41.0,H-LC,1.56,152875.0,-33220.0,80703.0,0.79,-17.85,52.79,25.51,15.0,-0.41,1.08,12.66,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.16,40.0,X-MC,3.27,306642.0,-21349.0,127870.0,0.81,-6.51,41.70,32.48,92.0,-0.17,2.16,4.17,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.38,59.0,X-LC,3.81,242623.0,8479.0,40154.0,-1.29,3.62,16.55,20.77,86.0,0.21,1.71,10.79,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49012.0,-15994.0,166920.0,-1.21,-24.60,340.57,232.18,198.0,-0.10,0.34,0.31,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,38.0,X-MC,7.53,80330.0,-3180.0,23207.0,1.07,-3.81,28.89,23.98,82.0,-0.14,0.56,1.79,X40,ATH,APPARELS
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239832.0,-21787.0,140997.0,-0.40,-8.33,58.79,45.57,7.0,-0.15,1.69,2.05,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.18,217580.0,-45785.0,155113.0,0.40,-17.38,71.29,41.51,84.0,-0.30,1.53,2.10,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49012.0,-15994.0,166920.0,-1.21,-24.60,340.57,232.18,198.0,-0.10,0.34,0.31,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,38.0,X-MC,7.53,80330.0,-3180.0,23207.0,1.07,-3.81,28.89,23.98,82.0,-0.14,0.56,1.79,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.75,40.0,X-SC,4.20,80718.0,-64442.0,140369.0,-0.49,-44.39,173.90,52.31,136.0,-0.46,0.57,8.05,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,44.0,X-MC,3.06,88185.0,-2493.0,17399.0,2.12,-2.75,19.73,16.44,101.0,-0.14,0.62,16.87,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.55,284632.0,-61324.0,128027.0,1.21,-17.73,44.98,19.28,1.0,-0.48,2.00,5.22,X40,ATH,IT
39,INFY,2275.00,-17.09,61.0,X-LC,4.86,323504.0,10616.0,161073.0,2.84,3.39,49.79,54.87,3.0,0.07,2.28,11.96,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,0.81,198583.0,-1555.0,22440.0,0.51,-0.78,11.30,10.44,4.0,-0.07,1.40,4.74,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,5.64,299286.0,-16556.0,129232.0,-0.23,-5.24,43.18,35.67,5.0,-0.13,2.10,7.89,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239832.0,-21787.0,140997.0,-0.40,-8.33,58.79,45.57,7.0,-0.15,1.69,2.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.35,76915.0,-36634.0,76684.0,2.74,-32.26,99.70,35.27,268.0,-0.48,0.54,88.25,XR,NTT,HOTELS
7,ASIANTILES,137.00,7416.67,64.0,L-SC,13.03,82465.0,-11345.0,87965.0,0.85,-12.09,106.67,81.67,269.0,-0.13,0.58,59.70,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,50.0,H-SC,7.97,92100.0,-5880.0,27483.0,-0.11,-6.00,29.84,22.05,152.0,-0.21,0.65,33.87,XR,ATH,FINANCE
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.47,123757.0,-1540.0,60468.0,-2.40,-1.23,48.86,47.03,141.0,-0.03,0.87,54.96,MH,ATH,POWER
70,SURYODAY,240.00,51.32,38.0,H-SC,14.53,139689.0,-39382.0,105116.0,-4.42,-21.99,75.25,36.71,135.0,-0.37,0.98,38.60,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.35,61.0,H-SC,4.82,148473.0,-34302.0,145370.0,-0.98,-18.77,97.91,60.77,125.0,-0.24,1.04,33.71,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,21.43,67.0,X-SC,6.44,215332.0,24326.0,33376.0,0.37,12.74,15.50,30.21,157.0,0.73,1.51,36.28,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7416.67,64.0,L-SC,13.03,82465.0,-11345.0,87965.0,0.85,-12.09,106.67,81.67,269.0,-0.13,0.58,59.70,XR,NTT,CERAMICS
6,ASIANPAINT,3465.66,-7.44,74.0,X-LC,5.62,233781.0,-17987.0,71186.0,1.64,-7.14,30.45,21.13,27.0,-0.25,1.64,25.75,X40,ATH,PAINTS
68,SONACOMS,806.63,-31.14,68.0,M-SC,6.26,85750.0,-15409.0,55412.0,-0.22,-15.23,64.62,39.54,202.0,-0.28,0.60,20.92,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.58
2,50,75.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.61
LC    33.27
MC    23.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.95
X40      16.40
X40N     12.43
XY25     11.93
XR        9.78
AR        7.75
OX40N     6.58
X200      1.79
X5K       1.48
SR        1.02
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.07
X-LC    22.55
X-MC    18.70
M-SC    12.53
X-SC     5.56
M-LC     5.17
H-LC     4.56
H-MC     2.32
M-MC     1.76
L-SC     1.45
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-4.67,38.51
IT,12.55,-17.83,80.90
FINANCE,12.16,-9.35,58.91
MISC,7.73,-18.14,77.73
PAINTS,5.72,-13.38,30.28
ELECTRICAL,5.69,-11.94,51.62
BANKS,3.97,-19.51,89.03
INSURANCE,3.94,1.30,35.50
AUTO,2.70,-49.56,111.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3312745.0,22
XR,1266164.0,13
AR,1185511.0,9
X40,824573.0,12
X40N,701637.0,9
OX40N,549554.0,9
XY25,476959.0,8
SR,275085.0,2
X5K,106896.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407075.0,24
M-SC,1261698.0,15
X-MC,1212997.0,14
X-LC,1073442.0,13
X-SC,557937.0,6
H-LC,291119.0,3
M-LC,273826.0,4
H-MC,258641.0,2
L-SC,254839.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1183855.0      6
           AR         840270.0      5
           XR         793955.0      7
M-SC       XY24       738152.0      6
X-MC       XY24       575612.0      4
X-LC       X40        442963.0      5
           XY24       311867.0      2
X-SC       X40N       304198.0      4
X-MC       X40        294791.0      6
H-SC       SR         275085.0      2
           OX40N      253442.0      3
X-LC       X40N       237469.0      3
H-LC       AR         221700.0      2
H-MC       XY24       184042.0      1
X-MC       XY25       182624.0      2
X-SC       XY24       166920.0      1
L-SC       XR         164649.0      2
X-MC       X40N       159970.0      2
M-MC       XY24       142286.0      1
M-SC       XR         138991.0      2
           OX40N      131323.0      4
           XY25       129691.0      1
           AR         123541.0      2
M-LC       XR         111522.0      1
           X5K        106896.0      1
L-SC       OX40N       90190.0      1
X-SC       X40         86819.0      1
X-LC       XY25        81143.0      3
H-MC       OX40N       74599.0      1
H-LC       X200        69419.0      1
H-SC       MH          60468.0      1
L-MC       XR          57047.0      1
M-LC       XY25        45397.0      1
L-LC       XY25        38104.0      1
M-LC       XY24        10011.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
